In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import datetime
import random
import collections
import itertools

import seaborn
import matplotlib.pyplot as plt

from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.svm import *
from sklearn.kernel_ridge import *
from sklearn.neighbors import
from sklearn.tree import 

from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.pipeline import *
from sklearn.decomposition import *
from sklearn.impute import

from imblearn.pipeline import 
from imblearn.over_sampling import 
from imblearn.under_sampling import
from imblearn.metrics import

from mlxtend.classifier import
import lightgbm as lgb
from lgb import 
from xgboost import 

import scipy
from scipy.stats import 
from scipy.special import


pd.set_options()
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
train = pd.read_csv('')
test = pd.read_csv('')

In [ ]:
#find numeric features
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric = []
for i in train.columns:
    if train[i].dtype in numeric_dtypes:
        numeric.append(i)
        
fig, axs = plt.subplots(ncols=2, nrows=0, figsize=(12, 120))
plt.subplots_adjust(right=2)
plt.subplots_adjust(top=2)
sns.color_palette('husl', 8)
for i, feature in enumerate(list(train[numeric]), i):
    plt.subplot(len(list(numeric)), 3, i)
    sns.scatterplot(x=feature, y=train['target'], hue='target', palette='Blues', data=train)
    plt.xlabel('{}'.format(feature), size=15, labelpad=12.5)
    plt.ylabel('target', size=15, labelpad=12.5)
    
    for j in range(2):
        plt.tick_params(axis='x', labelsize=12)
        plt.tick_params(axis='y', labelsize=12)
        
    plt.legend(loc='best', prop={'size': 10})
    
plt.show()

In [ ]:
corr = train.corr()
plt.subplots(figsize=(15, 12))
sns.heatmap(corr, vmax=0.9, cmap='Blues', square=True)

#Let's plot how our targets relates to some of the features in the dataset

In [ ]:
data =pd.concat([train['target'], train['dob']], axis=1)
f, ax=plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=train['gender'], y=train['target'], data=data)
fig.axis(ymin=0, ymax=1);

In [ ]:
#split features and labels
train_labels = train['targets'].reset_index(drop=True)
train_features  train.drop(columns=['target'], axis=1)
test_features = test

#combine train and test features
all_features = pd.concat([train_features, test_features]), reset_index(drop=True)
all_features.shape

handle columns(dob, gender, tags etc)

In [ ]:
all_features['gender'] = all_features['gender'].replace(['Male', 'male', 'Male ','Male  ', '  ', '?????'], 'Male')
all_features['gender'] = all_features['gender'].replace(['Female', 'Female ', 'Female  ', 'Female   ', 'Female    '], 'Female')

all_features['primary_tags'] = all_features['primary_tags'].fillna('[]')
all_features['primary_tags'] = all_features['primary_tags'].apply(literal_eval)
all_features['primary_tags'] = all_features['primary_tags'].str.get('primary_tags')


all_features['vendor_tag'] = all_features['vendor_tag'].fillna('[]')
all_features['vendor_tag_name'] = all_features['vendor_tag_name'].fillna('[]')

def to_list(string):
    ls = list(string.split(','))
    return ls

all_features['vendor_tag'] = all_features['vendor_tag'].apply(to_list)
all_features['vendor_tag_name'] = all_features['vendor_tag_name'].apply(to_list)

missing values

In [ ]:
# determine the threshold for missing values
def percent_missing(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: round(data[df_cols[i]].isnull().mean()*100,2)})
    
    return dict_x

missing = percent_missing(all_features)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Percent of missing data')
df_miss[0:100]

In [ ]:
# Visualize missing values
sns.set_style("white")
f, ax = plt.subplots(figsize=(8, 7))
sns.set_color_codes(palette='deep')
missing = round(train.isnull().mean()*100,2)
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar(color="b")
# Tweak the visual presentation
ax.xaxis.grid(False)
ax.set(ylabel="Percent of missing values")
ax.set(xlabel="Features")
ax.set(title="Percent missing data by feature")
sns.despine(trim=True, left=True)

In [ ]:
# Some of the non-numeric predictors are stored as numbers; convert them into strings 
all_features['status_x'] = all_features['status_x'].apply(str)
all_features['verified_x'] = all_features['verified_x'].astype(str)


In [ ]:
#fixed skewed features
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric = []
for i in all_features.columns:
    if all_features[i].dtype in numeric_dtypes:
        numeric.append(i)

In [ ]:
# Create box plots for all numeric features
sns.set_style("white")
f, ax = plt.subplots(figsize=(8, 7))
ax.set_xscale("log")
ax = sns.boxplot(data=all_features[numeric] , orient="h", palette="Set1")
ax.xaxis.grid(False)
ax.set(ylabel="Feature names")
ax.set(xlabel="Numeric values")
ax.set(title="Numeric Distribution of Features")
sns.despine(trim=True, left=True)

In [ ]:
# Find skewed numerical features
skew_features = all_features[numeric].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

print("There are {} numerical features with Skew > 0.5 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skew_features.head(10)

In [ ]:
# Normalize skewed features
for i in skew_index:
    all_features[i] = boxcox1p(all_features[i], boxcox_normmax(all_features[i] + 1))

In [ ]:
# Let's make sure we handled all the skewed values
sns.set_style("white")
f, ax = plt.subplots(figsize=(8, 7))
ax.set_xscale("log")
ax = sns.boxplot(data=all_features[skew_index] , orient="h", palette="Set1")
ax.xaxis.grid(False)
ax.set(ylabel="Feature names")
ax.set(xlabel="Numeric values")
ax.set(title="Numeric Distribution of Features")
sns.despine(trim=True, left=True)

In [ ]:
create new features

In [ ]:
#feature transformation
def logs(res, ls):
    m = res.shape[1]
    for l in ls:
        res = res.assign(newcol=pd.Series(np.log(1.01+res[l])).values)   
        res.columns.values[m] = l + '_log'
        m += 1
    return res

log_features = [num_columns]

all_features = logs(all_features, log_features)

In [ ]:
def squares(res, ls):
    m = res.shape[1]
    for l in ls:
        res = res.assign(newcol=pd.Series(res[l]*res[l]).values)   
        res.columns.values[m] = l + '_sq'
        m += 1
    return res 

squared_features = [num_columns]
all_features = squares(all_features, squared_features)

In [ ]:
#encode categorical features

In [ ]:
#recreate training and test sets
X = all_features.iloc[:len(train_labels), :]
X_test = all_features.iloc[len(train_labels):, :]
X.shape, train_labels.shape, X_test.shape